In [17]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import Dataset

In [18]:
# set your file dir path
npy_directory = '/bask/projects/p/phwq4930-renal-canc/Zeyu/WSI_Dataset/Conch/TCGA-*/'
txt_directory = '/bask/projects/p/phwq4930-renal-canc/Zeyu/WSI_Dataset/TCGA_Report/TXT-GPT4o'
patients_file = '/bask/projects/p/phwq4930-renal-canc/Zeyu/WSI_Dataset/TCGA_Clinical.tsv'
patients_list = pd.read_csv(patients_file, sep='\t')[['case_submitter_id','project_id']].drop_duplicates()
patients_list.columns = ['patient_id','project']

In [25]:
feature1_files = glob.glob(os.path.join(npy_directory, "*_0_1024.npy"))
feature2_files = glob.glob(os.path.join(npy_directory, "*_1_512.npy"))
feature3_files = glob.glob(os.path.join(npy_directory, "*_1_1024.npy"))
text_files = glob.glob(os.path.join(txt_directory, "*.txt"))
df_fea1 = pd.DataFrame(feature1_files, columns=['fea1_file_path'])
df_fea2 = pd.DataFrame(feature2_files, columns=['fea2_file_path'])
df_fea3 = pd.DataFrame(feature3_files, columns=['fea3_file_path'])
df_text = pd.DataFrame(text_files, columns=['text_file_path'])
df_fea1['slide_id'] = df_fea1['fea1_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea2['slide_id'] = df_fea2['fea2_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea3['slide_id'] = df_fea3['fea3_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])

In [26]:
df_fea1['slide_id'] = df_fea1['fea1_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea2['slide_id'] = df_fea2['fea2_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea3['slide_id'] = df_fea3['fea3_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])

df_fea1 = df_fea1.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)
df_fea2 = df_fea2.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)
df_fea3 = df_fea3.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)

df_fea = pd.merge(df_fea1, df_fea2, on='slide_id', how='inner')
df_fea = pd.merge(df_fea, df_fea3, on='slide_id', how='inner')

df_fea['patient_id'] = df_fea['slide_id'].apply(lambda x: x[:12])

df_text['patient_id'] = df_text['text_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])

df = pd.merge(df_fea, df_text, on='patient_id', how='inner')

df = pd.merge(df, patients_list, on='patient_id', how='inner')

# df['project_old'] = df['fea1_file_path'].apply(lambda x: x.split('/')[-2])

In [28]:
data = []
for i in tqdm(range(df.shape[0])):
#     feature1_content = np.load(df.iloc[i]['fea1_file_path'], allow_pickle=True) 
#     feature1 = feature1_content[()]['feature'].cpu().numpy().flatten() # Nx512
#     feature1_cor = np.array([x.split('_')[:2] for x in feature1_content[()]['index']]).astype('int').flatten() # Nx2
    
#     feature2_content = np.load(df.iloc[i]['fea2_file_path'], allow_pickle=True)
#     feature2 = feature2_content[()]['feature'].cpu().numpy().flatten()
#     feature2_cor = np.array([x.split('_')[:2] for x in feature2_content[()]['index']]).astype('int').flatten()
    
#     feature3_content = np.load(df.iloc[i]['fea3_file_path'], allow_pickle=True)
#     feature3 = feature3_content[()]['feature'].cpu().numpy().flatten()
#     feature3_cor = np.array([x.split('_')[:2] for x in feature3_content[()]['index']]).astype('int').flatten()
    with open(df.iloc[i]['text_file_path'], 'r') as txt_file:
        txt_content = txt_file.read()
        
    data.append({'f1024': df.iloc[i]['fea1_file_path'], 
                 'f2048': df.iloc[i]['fea2_file_path'], 
                 'f4096': df.iloc[i]['fea3_file_path'],
                 'slide_id': df['slide_id'].iloc[i],
                 'project': df['project'].iloc[i],
                 'description': txt_content})

100%|██████████| 11664/11664 [00:19<00:00, 591.43it/s]


In [29]:
# to huggingface format
data = pd.DataFrame(data)

In [30]:
dataset = Dataset.from_pandas(data)

In [33]:
dataset.push_to_hub('CNX-PathLLM/TCGA-WSI-Description')

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/CNX-PathLLM/TCGA-WSI-Description/commit/3943c12f76614c8e41004404e37f2b21c95b249c', commit_message='Upload dataset', commit_description='', oid='3943c12f76614c8e41004404e37f2b21c95b249c', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
dataset

Dataset({
    features: ['f1024', 'f2048', 'f4096', 'slide_id', 'project', 'description'],
    num_rows: 11664
})

In [48]:
# 保存数据集
dataset.save_to_disk('/bask/projects/p/phwq4930-renal-canc/Zeyu/WSI_Dataset')
# 加载数据集
loaded_dataset = Dataset.load_from_disk('/bask/projects/p/phwq4930-gbm/Zeyu/WSI_Dataset/WVLMdata_test')

Saving the dataset (19/19 shards): 100%|██████████| 910/910 [00:04<00:00, 189.66 examples/s]


In [49]:
loaded_dataset

Dataset({
    features: ['f1', 'cor1', 'f2', 'cor2', 'f3', 'cor3', 'label'],
    num_rows: 910
})

In [1]:
# upload a sample dataset to huggingface
from datasets import load_dataset, concatenate_datasets, load_from_disk

/bask/projects/p/phwq4930-gbm/Zeyu/asiw9691_conda_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dirs = '/bask/projects/p/phwq4930-gbm/Zeyu/WSI_Dataset/TCGA-WSI-Text-Folds'
dataset = load_from_disk(load_dirs)

In [3]:
dataset = dataset[f'fold_9']
# dataset.push_to_hub("CNX-PathLLM/TCGA-WSI-Text-Sample")

In [4]:
dataset

Dataset({
    features: ['f1', 'cor1', 'f2', 'cor2', 'f3', 'cor3', 'label', 'slide_id', 'project'],
    num_rows: 1247
})